# Final Notebook-Comprehensive Notebook detailing the process to the final model and project overview.

# Overview of Project
 This project aims to utilize Illinois 2018 American Community Survey data to create an imcome predicitor to aid Illinois and governmental programs with their allocation of the programs. In particular programs such as SNAP and unemployment. This will allow for a faster application process due to better accuracy. This is very crucial in especially when looking at unemployment. This has been one of the most vital neccesities of this COVID-19 age due to the massive amount of layoff/furloughs done by organizations. In creating a better system to access these individuals and their income we can better deploy these services to the citizens that need them. More information on this topic can be found [here]
 
# Data acquisition 
 Prior to running any of the Jupyter Notbooks or data in this download the neccesary files and create the environments found within them.

Download [Andaconda](https://docs.anaconda.com/anaconda/install/) to be able to install the environment and use your preference of commandline options or Editor to view the data.
Also go to this [Census Link](https://www2.census.gov/programs-surveys/acs/data/pums/2018/5-Year/) and down load the csv_hil.zip to acquires the data used in this project.

### From the list options below choose the environments within this neccesary envrionment for your operating system.

[`Capstone.yml`]

To utilize Geopandas:

[`geopandas.yml`]

All files are located in the main directory and src folder.
Run these two lines of code after the download to apply the kernals to your IDE.

`python -m ipykernel install --user --name tens --display-name "Python 3 (tens)"` 

`python -m ipykernel install --user --name Capstone --display-name "Python 3 (Capstone)"`  

 
# Goals:
This project aims to:
- Build a modle that predicts an individuals income based on multiple parameters such as property value, proterty taxes and number of people in a household.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.js_functions import training_data
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from src.js_functions import  process_data
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

**Importing Data**

In [12]:
df=pd.read_csv("../../Data/csv_hil/psam_h17.csv")

**The orginal dataframe will now processed via the process data function. For more about the cleaning and processing of the data [check out this notbook](../Explortatory/Data_cleaning_processing.ipynb) as well as [this py file](../../src/js_functions.py).**


In [13]:
X_train, X_test,y_train, y_test=process_data(df)

**Now that the data is proccessed and loaded let run our initial model.**

In [14]:
#This is our baseline model.
lr=LinearRegression()

In [29]:
lr.fit(X_train,y_train)

LinearRegression()

In [16]:
lr.score(X_train,y_train)

0.3847594087743529

In [17]:
scores= cross_val_score(lr,X_train,y_train,cv=5)

In [18]:
scores.mean()

0.38422207389115826

**Basline model has a R-squared score of .38 so not to bad as basline model. Let's see if we can increase this value through a different algorithm. Overall, This initial model is telling that 38% of the variance seen in the target columns is due to the current features we are using.**

**Lets run our Random forest regressor seeing as this was the best model from the model iteration process. For a more detail dive into this open [KNN notebook](../Explortatory/KNN_regressor.ipynb), [Neural Networks](../Explortatory/Neural_networks.ipynb), [Random Forest Regressor notebook](../Explortatory/Random_forest_regressor.ipynb)**

In [19]:
rfr=RandomForestRegressor(max_depth=10, min_samples_split= 4, n_estimators = 100)

In [20]:
rfr.fit(X_train,y_train)

RandomForestRegressor(max_depth=10, min_samples_split=4)

In [21]:
rfr.score(X_train,y_train)

0.4707455075369493

In [22]:
scores= cross_val_score(rfr,X_train,y_train,cv=5)

In [23]:
scores.mean()

0.4071296998755914

**Lets see how our final model preforms on the test data set for a final evaluation.**

In [24]:
rfr.score(X_test,y_test)

0.41705033057573215

In [25]:
scores1= cross_val_score(rfr,X_test,y_test,cv=5)

In [26]:
scores1.mean()

0.4074608004753443

 **The model performs well on the test data where the level of vairance association is not to far off from the training data. With some more iteration we may be able to increase this value and create are more accurate model**

In [27]:
y_hat=rfr.predict(X_test)

In [55]:
print("Actual=%s, Predictied=%s" % (y_test.values.mean(),y_hat.mean()))

Actual=86708.29873633395, Predictied=86634.44640730703


**From this we can see that our actual value  and predicted value average do not differ to much which makes sense due to the model trying to acomodate all the data**

In [58]:
print("Actual=%s, Predictied=%s" % (y_test.values[0],y_hat[0]))

Actual=35000.0, Predictied=64088.77956877114


**However, when we look at the individual values and their predicted income counterparts there is a large discrepancy in some case such as the first value of the test data being 35,000 and the predcited value being 64,000. This is something I expected due to combining all of the counties in one dataframe. This is due to the cost of living and overall socio-economic advantages or disadvantages of a county based on it geographical location**

### End analysis conlusion:
- The final model predicts about 40% of the vairiance seen within the data. 
- When looking at this from a quantitative standpoint there is about ~70 dollar average differece in the actual and predicted values with this model. However, when you look at individual values there is about a 29,000 dollar differnce in those values.

### Next steps:
- I would like to continue to Iterate on the current model and reduce the amount of vairance seen between the actual and predicted values.
- I would also like to subset the counties and see how that would affect the predicted values due to haveing more geographically related data.
- Moreover, I would like to do some more feature enginerring to possiblty increase the R-Squared value.
- Lastly, I would like to find more recent census data to have a better estimate to the current day household income.